In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from walks import Route, Point, Segment

In [ ]:
from collections import defaultdict
from datetime import date
import statistics

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
from db_routes import load_routes_from_db

In [ ]:
requested_map = cimgt.OSM()  # Open Street Map

In [ ]:
routes = load_routes_from_db()

route_bounds = (1000, 1000, -1000, -1000)

from_date = date(2020, 10, 1)
print(f'applying {from_date=}')
unwanted_route_ids = [rt_id for rt_id, rt in routes.items() if rt.walk_date < from_date]
for rt_id in unwanted_route_ids:
    routes.pop(rt_id)

for rt in routes.values():
    print(rt.walk_date, type(rt.walk_date))
    bnds = rt.bounds
    route_bounds = (
        min(route_bounds[0], bnds[0]),
        min(route_bounds[1], bnds[1]),
        max(route_bounds[2], bnds[2]),
        max(route_bounds[3], bnds[3]),
    )

print(f'{len(routes)} routes loaded for mapping')

In [ ]:
def locs_at(mins):
    pts = [rt.point_at(mins * 60) for rt in routes.values()]

    lons = [pt.lon for pt in pts if pt]
    lats = [pt.lat for pt in pts if pt]

    return (lons, lats)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(projection=requested_map.crs))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

ax.add_image(requested_map, 14)

lons, lats = locs_at(15)
plt.scatter(lons, lats, color='aqua', s=100, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(30)
plt.scatter(lons, lats, color='indigo', s=100, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(45)
plt.scatter(lons, lats, color='red', s=100, alpha=1, transform=ccrs.PlateCarree())

avg_start_lon = statistics.mean(rt.points[0].lon for rt in routes.values())
avg_start_lat = statistics.mean(rt.points[0].lat for rt in routes.values())
plt.scatter([avg_start_lon], [avg_start_lat], color='lime', s=1000, alpha=1, marker='o', transform=ccrs.PlateCarree())

plt.title(f'After 15, 30, 45 minutes')

plt.show()

In [ ]:
print(f'considering {len(routes)} routes')
points = [pt for rt in routes.values() for pt in rt.points]
print(f'extracted {len(points)} points')

In [ ]:
lons = [p.lon for p in points]
lats = [p.lat for p in points]
eles = [p.ele for p in points]
min_ele = min(eles)
max_ele = max(eles)

In [ ]:
def col_for(e, min_e, max_e):
    index = (e - min_e) / (max_e - min_e)
    return (index, index, index)

cols = [col_for(e, min_ele, max_ele) for e in eles]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(projection=requested_map.crs))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

ax.add_image(requested_map, 14)

plt.scatter(lons, lats, color=cols, s=50, alpha=1, transform=ccrs.PlateCarree())

plt.title(f'Elevations')

plt.show()

In [ ]:
print(len(lons), len(lats), len(cols), len(points))